In [1]:
from scipy.stats import chi2, norm # for tests

import matplotlib.pyplot as plt # for graphs
import numpy as np # for matrix operations
import pandas as pd # for DataFrames
import statsmodels.api as sm # for statistics
import time # for getting the time of the cell execution
import scipy.stats as stats # for statistical distributions
from scipy.stats import chi2 # to get chi-squared distribution

## Problem 2 "Exchange rates again"

Recall the problem "Testing for forecast unbiasedness" from Assignment 3. For both types of forwards, test the conditional unbiasedness hypothesis by using the overlapping block bootstrap with the block length equal 15. With 3-month forwards, use the Newey- West HAC long run variance estimator. Compare the critical values you obtained using the asymptotic and bootstrap approaches. In case of large discrepancies, speculate on their possible sources.

Task 2, HA 3:{

The data file FwdSpot1.dat contains monthly spot and 1-month forward exchange rates, the datafilee FwdSpot3.dat- monthly spot and 3-month forward exchange rates, in $/foreign currency, for the British Pound, French Franc and Japanese Yen, for 1973:3 to 1992:8 (234 observations). Each row contains the month, the year, the spot rates for Pound, Franc, and Yen, and then the forward rates for the same three currencies. Download the data, then take logarithms of the rates.ces.

We are interested in testing the conditional unbiasedness hypothesis that:

$E_t[st+k] = f_{t,k}$

where:
- $s_t$ is the spot rate at time t
- $f_{t,k}$ is the forward rate for k-month forwards at time t
- $E_t$ denotes the mathematical expectation conditional on time t information

The statement above says that the forward rate is a conditionally unbiased predictor of the future spot exchange rate.

To test this theory, one nests (1) within the following econometric model:

$s_{t+k} - s_t = β + γ (f_{t,k} - s_t) + ε_{t+k}$ 

$E_t[ε_{t+k}] = 0$

and test H0: β = 0; γ = 1.

The current spot rate is subtracted to achieve stationarity.
The difference $s_{t+k} - s_t$ is called the exchange rate depreciation, the difference $f_{t,k} - s_t$ the
forward premium. 

For the three currencies and both types of forwards, estimate (2) by OLS
and test for conditional unbiasedness. Do not forget HAC variance estimation whenever
appropriate; explain why it is needed or not needed. Discuss the test results.}

### OLS model

So, our model for the OLS is the following:

$ed_{t+k} = \beta + \gamma fp_{t+k} + \epsilon_{t+k}$

### Beta:

$\hat{\beta} = (X' X)^{-1} X' Y$

$\hat{\beta}$ is the vector of estimated coefficients.

$X$ is the matrix of predictor variables (also known as the design matrix).

$Y$ is the vector of the target values.

### Covariance matrix (White)

To get estimated covariance matrix (White estimator):

$\hat{e} = \hat{Y} - X * \hat{\beta}'$

$\hat{V}_{\beta} = \hat{Q}_{xx}^{-1}\hat{V}_{xe}\hat{Q}_{xx}^{-1}$; This is our covariance matrix.

$\hat{V}_{xe} = \frac{1}{n}X' \Omega X$, where $\Omega$ is a diagonal matrix with error term $\hat{e_{i}^{2}}$ on the $i_{}^{th}$ place.

$\hat{Q}_{xx}^{-1} = (\frac{1}{n}X'X)_{}^{-1}$

### Covariance matrix (HAC: Newly-West)

To get estimated covariance matrix (HAC: Newly-West):

$Z = X\hat{e}$

lags: i = 1, ..., T-1

$\hat{\Gamma}_j = \frac{1}{T} \sum_{t = max(1,1+j)}^{t = min(1,1+j)} (Z_t - \overline{Z}_t)(Z_{t-j} - \overline{Z}_t)'\xrightarrow{p} \Gamma_j$

then the estimator itself:

$\hat{V}_z^{NW} = \sum_{j=-m}^{m}(1-\frac{|j|}{m+1})\hat{\Gamma}_j$

where

m = $\lfloor 4(\frac{T}{100}_{}^{1/3})  \rfloor$

$\lfloor$ - integer part

Get the standard deviation:

$se(\hat{\beta_j}) = \sqrt{\frac{1}{n}[\hat{V_{\beta}}]_{jj}}$

In [2]:
#OLS, White estimator, Newly-West estimator are implemented in the following module
import sys

path = 'C:/Users/Popov/Documents/NES_studies/Python/NES_Helper' # Местоположение файла на диске
sys.path.append(path)
# this module was expended in the 3 HA
from NES_helper import Estimators as est

### Wald test

$t_w = nh(\hat{\beta})'(\hat{H}\hat{V}_{\beta}\hat{H}')_{}^{-1}h(\hat{\beta})$

$ h = H \beta - q$, q- restriction RHS

Note that 

$ed_{t+k} = \beta + \gamma fp_{t+k} + \epsilon_{t+k}$

$H_0$ : β = 0; γ = 1.

Then H = [[0, 1], [1, 0]] - both hypothesis ([0, 1] - γ, [1, 0] - \beta). Thus h = [1,0] (γ = 1, β = 0)

In [3]:
def Wald_test(beta_hat, V_hat, H, q):
    n = X.shape[0]
    h = H @ beta_hat - q
    W = n * h.T @ np.linalg.inv(H @ V_hat @ H.T) @ h
    # pv2 = chi2.sf(W, H.shape[0])
    return W

### Bootstrap Wald statistic:

$$h = H \beta - q = \hat{\beta}_b^{*} - \hat{\beta}, $$ then 

$$ \{W_b^{*} = n(\hat{\beta}_b^{*} - \hat{\beta})'(\hat{V}_{\beta b}^{*})_{}^{-1}(\hat{\beta}_b^{*} - \hat{\beta})\}_{b=1}^{B}$$

$\hat{\beta}_b^{*}$ - bootstrap beta estimate (in the code - Beta_w)

$\hat{V}_{\beta b}^{*}$ - bootstrap covarriance matrix estimate.

$\hat{\beta}$ - OLS beta estimate (in the code - Beta0)

Notation: B = N (number of bootstrap iterations).

In [4]:
def Bootstrap_Wald(X, Y, N, l):
    '''
    X - regressors' matrix;
    Y - dependent variable's bector;
    N - number of bootstrap repetitions;
    l - block size
    '''
    n=X.shape[0] # size of the sample
    T = n // l * l #largest multiple of l that does not exceed n
    Beta0 = np.linalg.inv(X.T @ X / n) @ (X.T @ Y / n) # those are asymptotic estimates (OLS) on the 
    # initial sample
    Wb = np.zeros(N)

    # Precompute all possible blocks with len l
    all_blocks_X = [X[i:i + l] for i in range(T - l + 1)]
    all_blocks_Y = [Y[i:i + l] for i in range(T - l + 1)]

    for i in range(N): # initialize bootstrap loop
        # Randomly sample blocks with replacement
        sampled_indices = np.random.randint(0, len(all_blocks_X), size=T // l)
        # 0 - starting index for sampling
        # len(all_blocks_X) -  total number of possible blocks
        # size=T // l - number of non-overlapping blocks of size l that fit within T
        #(number of random integers to generate). Not really determined-
        # could draw another number of blocks on each iteration.
        Xsamp = np.vstack([all_blocks_X[j] for j in sampled_indices])
        Ysamp = np.concatenate([all_blocks_Y[j] for j in sampled_indices])
        # Xsamp: Stacks blocks of X data vertically to create a larger
        # matrix where each row represents a data point in the bootstrap sample.
        #Ysamp: Concatenates blocks of Y data into a single larger array,
        # maintaining the 1-dimensional structure as Y is already a vector.
        Wb[i] = Wald_b(Xsamp, Ysamp, Beta0)
    return np.quantile(Wb, 0.95)

def Wald_b(Xsamp, Ysamp, Beta0):
  # Compute Estimates
    n=Xsamp.shape[0]
    QxxInv = np.linalg.inv(Xsamp.T @ Xsamp / n)
    Beta_w = QxxInv @ (Xsamp.T @ Ysamp / n) 
    
    # Compute covariance matrix
    ErHat2 = (Ysamp - Xsamp @ Beta_w)**2 
    Omega = np.diag(ErHat2.reshape(n))
    Vxe = (Xsamp.T @ Omega @ Xsamp)/ n
    Vb = QxxInv @ Vxe @ QxxInv
    
    # Compute Bootstrap Wald statistics
    h = Beta_w - Beta0 # our new restriction: Beta_w- beta estimate of OLS on Bootstrap generated population,
    # Beta0 - beta estimate of OLS on the initial sample
    H = np.array([[1, 0], [0, 1]])
    W = n * h.T @ np.linalg.inv(H @ Vb @ H.T) @ h

    return W

### k = 1 (lags) - One month prediction

Econometric model:

$s_{t+k} - s_t = β + γ (f_{t,k} - s_t) + ε_{t+k}$

Note that as we predit only for one period, there is no oberlapping of predictions => no autocorrelations => use simple White estimator for variance.

In [5]:
data = pd.read_csv('FwdSpot1.dat', header=None,
                    sep=' ', skipinitialspace=True)
# skipinitialspace=True - to exclude the leading spaces - because otherwise
# we a column of nans and a colun of dates
data

,0,1,2,3,4,5,6,7
0,3,73,2.4755,0.2203,0.003752,2.469621,0.220649,0.003780
1,4,73,2.4869,0.2187,0.003763,2.482403,0.218873,0.003766
2,5,73,2.5720,0.2305,0.003788,2.568699,0.230600,0.003803
3,6,73,2.5825,0.2410,0.003805,2.578110,0.241000,0.003834
4,7,73,2.5072,0.2424,0.003772,2.501225,0.242823,0.003823
...,...,...,...,...,...,...,...,...
229,4,92,1.7793,0.1801,0.007513,1.769202,0.179159,0.007508
230,5,92,1.8315,0.1853,0.007834,1.822495,0.184383,0.007829
231,6,92,1.9110,0.1966,0.008019,1.900904,0.195526,0.008013
232,7,92,1.9190,0.2004,0.007852,1.908094,0.199241,0.007847


Note that date is in the descending order. (column 1)

In [6]:
# clean the data
data = data.iloc[:, 2:]
# rename columns according to the task
data.rename(columns={2: 'BP', 3: 'FF', 4: 'JY', 5: 'BP_f', 6: 'FF_f', 7: 'JY_f'}, inplace=True)
# take logarithms
data = data.apply(np.log)
data.head(3)

,BP,FF,JY,BP_f,FF_f,JY_f
0,0.906442,-1.512765,-5.585466,0.904065,-1.511182,-5.578031
1,0.911037,-1.520054,-5.582539,0.909227,-1.519264,-5.581742
2,0.944684,-1.467504,-5.575917,0.943400,-1.467071,-5.571965


For our coefficients estimate we will use standard OLS (as they ask us in the task).

In [7]:
# let's at first do only for the 1-lag
k = 1
# note, that in our dataset previous years are at the bottom
# lagged spot prices for pound, franc and yen.

s_t_k = data.iloc[:, :3]

# current values
s_t = s_t_k.shift(-k)
# calculate the exchange rate depreciation
e_d = (s_t - s_t_k.values).dropna().values
# current futures values
f_t_k = data.iloc[:, 3:]
# now calculate the forward premium
f_p = (f_t_k - s_t_k.values).iloc[:e_d.shape[0]].values

In [8]:
%%time
coef_names = [['alpha_1', 'beta_1'], ['alpha_2', 'beta_2'], ['alpha_3', 'beta_3']]
names = data.columns.tolist() 

# Loop through OLS (for each currency)
for i, name in enumerate(coef_names):
    print("OLS for " + names[i])
    
    X = sm.add_constant(f_p[:, i])
    Y = e_d[:, i]
    
    # Initialize the Estimators class with data
    esti = est(X, Y)
    
    # Estimate beta
    beta_hat = esti.beta_est()
    
    # Calculate White standard errors
    V_hat = esti.White_est(beta_hat)
    SDs = esti.SD(V_hat)
    
    # Perform Wald test
    w = Wald_test(beta_hat, V_hat, np.array([[0, 1], [1, 0]]), np.array([1, 0]))
    df = np.array([[0, 1], [1, 0]]).shape[0] # number of restrictions is te degress of freedom for
    # chi-squared to which Wald statistic is converging
    
    # Perform bootstrap Wald test
    
    N = 100000 # number of repetitions
    l = 15 # block size
    wb = Bootstrap_Wald(X, Y, N, l)

    # Print coefficients with their standard errors
    for coef, coef_name, sd in zip(beta_hat, coef_names[i], SDs):
        print(f"{coef_name}: {round(coef, 4)} ({round(sd, 4)})")
    print('----  ----  ----  ----  ----')   
    print("Wald statistic" + "is:", round(w, 4), end="\n\n")
    print("Critical value (95% quantile of chi-squared) for Wald statistic", round(stats.chi2.ppf(0.95, df),4))
    print("Bootstrap Wald statistic 95% quantile (simulated critical value) " + "is:", round(wb, 4))
    print('--------------------------------------------------------------------------------------\n')

OLS for BP
alpha_1: -0.0023 (0.0024)
beta_1: -0.7261 (0.6401)
----  ----  ----  ----  ----
Wald statisticis: 7.4459

Critical value (95% quantile of chi-squared) for Wald statistic 5.9915
Bootstrap Wald statistic 95% quantile (simulated critical value) is: 13.6796
--------------------------------------------------------------------------------------

OLS for FF
alpha_2: -0.0023 (0.0026)
beta_2: -0.9606 (0.85)
----  ----  ----  ----  ----
Wald statisticis: 5.7265

Critical value (95% quantile of chi-squared) for Wald statistic 5.9915
Bootstrap Wald statistic 95% quantile (simulated critical value) is: 9.6387
--------------------------------------------------------------------------------------

OLS for JY
alpha_3: 0.0036 (0.0023)
beta_3: -0.1528 (0.5595)
----  ----  ----  ----  ----
Wald statisticis: 4.8988

Critical value (95% quantile of chi-squared) for Wald statistic 5.9915
Bootstrap Wald statistic 95% quantile (simulated critical value) is: 15.9921
---------------------------------

On 5% level the  is not rejected for franc, yen, but is rejected for pound)

### k = 3 (lags) - Three month prediction

In [9]:
data = pd.read_csv('FwdSpot3.dat', header=None,
                    sep=' ', skipinitialspace=True)
# skipinitialspace=True - to exclude the leading spaces - becuase otherwise
# we a column of nans
data

,0,1,2,3,4,5,6,7
0,3,73,2.4755,0.2203,0.003752,2.458357,0.221151,0.003840
1,4,73,2.4869,0.2187,0.003763,2.473906,0.219171,0.003782
2,5,73,2.5720,0.2305,0.003788,2.562098,0.230973,0.003843
3,6,73,2.5825,0.2410,0.003805,2.571008,0.241259,0.003886
4,7,73,2.5072,0.2424,0.003772,2.488521,0.243514,0.003919
...,...,...,...,...,...,...,...,...
229,4,92,1.7793,0.1801,0.007513,1.751900,0.177368,0.007502
230,5,92,1.8315,0.1853,0.007834,1.804800,0.182498,0.007821
231,6,92,1.9110,0.1966,0.008019,1.882100,0.193424,0.008004
232,7,92,1.9190,0.2004,0.007852,1.886600,0.196951,0.007840


In [10]:
# clean the data
data = data.iloc[:, 2:]
# rename columns according to the task
data.rename(columns={2: 'BP', 3: 'FF', 4: 'JY', 5: 'BP_f', 6: 'FF_f', 7: 'JY_f'}, inplace=True)
# take logarithms
data = data.apply(np.log)
data.head(3)

,BP,FF,JY,BP_f,FF_f,JY_f
0,0.906442,-1.512765,-5.585466,0.899493,-1.508910,-5.562283
1,0.911037,-1.520054,-5.582539,0.905798,-1.517903,-5.577502
2,0.944684,-1.467504,-5.575917,0.940826,-1.465454,-5.561502


In [11]:
# let's at first do only for the 1-lag
k = 3
# note, that in order to construct our model,
# we need to say that we ate in the period t+k. Then k-lag - period t.
# current spot prices for pound, franc and yen.

s_t_k = data.iloc[:, :3]

# lagged values
s_t = s_t_k.shift(-k)
# calculate the exchange rate depreciation
e_d = (s_t - s_t_k.values).dropna().values
# current futures values
f_t_k = data.iloc[:, 3:]
# now calculate the forward premium
f_p = (f_t_k - s_t_k.values).iloc[:e_d.shape[0]].values

Because k = 3 > 1(# of lags) => $e_t$ serially correlated. Thus, HAC is needed => Newey-West estimator (the standard one).

In [12]:
# Loop through OLS (for each currency)
coef_names = [['alpha_1', 'beta_1'], ['alpha_2', 'beta_2'], ['alpha_3', 'beta_3']]
names = data.columns.tolist()
for i, name in enumerate(coef_names):
    print("OLS for " + names[i])
    
    X = sm.add_constant(f_p[:, i])
    Y = e_d[:, i]
    
    # Initialize the Estimators class with data
    esti = est(X, Y)
    
    # Estimate beta
    beta_hat = esti.beta_est()
    
    # Calculate Newey-West standard errors
    V_hat = esti.NW_est(beta_hat)
    SDs = esti.SD(V_hat)
    
    # Perform Wald test
    w = Wald_test(beta_hat, V_hat, np.array([[0, 1], [1, 0]]), np.array([1, 0]))
    df = np.array([[0, 1], [1, 0]]).shape[0] # number of restrictions is te degress of freedom for
    # chi-squared to which Wald statistic is converging
    
    # Perform bootstrap Wald test
    
    N = 100000 # number of repetitions
    l = 15 # block size
    wb = Bootstrap_Wald(X, Y, N, l)

    # Print coefficients with their standard errors
    for coef, coef_name, sd in zip(beta_hat, coef_names[i], SDs):
        print(f"{coef_name}: {round(coef, 4)} ({round(sd, 4)})")
    print('----  ----  ----  ----  ----')   
    print("Wald statistic" + "is:", round(w, 4), end="\n\n")
    print("Critical value (95% quantile of chi-squared) for Wald statistic", round(stats.chi2.ppf(0.95, df),4))
    print("Bootstrap Wald statistic 95% quantile (simulated critical value) " + "is:", round(wb, 4))
    print('--------------------------------------------------------------------------------------\n')

OLS for BP
alpha_1: -0.0187 (0.0076)
beta_1: -2.0586 (0.7002)
----  ----  ----  ----  ----
Wald statisticis: 19.094

Critical value (95% quantile of chi-squared) for Wald statistic 5.9915
Bootstrap Wald statistic 95% quantile (simulated critical value) is: 22.4234
--------------------------------------------------------------------------------------

OLS for FF
alpha_2: -0.0044 (0.0091)
beta_2: -0.4804 (0.8427)
----  ----  ----  ----  ----
Wald statisticis: 3.7572

Critical value (95% quantile of chi-squared) for Wald statistic 5.9915
Bootstrap Wald statistic 95% quantile (simulated critical value) is: 28.9017
--------------------------------------------------------------------------------------

OLS for JY
alpha_3: 0.0131 (0.0071)
beta_3: -0.602 (0.4634)
----  ----  ----  ----  ----
Wald statisticis: 12.5791

Critical value (95% quantile of chi-squared) for Wald statistic 5.9915
Bootstrap Wald statistic 95% quantile (simulated critical value) is: 34.6868
------------------------------

$H_0$: forward rate is a conditionally unbiased predictor of the future spot exchange rate

On 5% level the $H_0$ is not rejected for franc, but is rejected for pound, yen. (3 months forecast)

recall the results for one month forecast:

On 5% level the $H_0$ is not rejected for franc, yen, but is rejected for pound. (1 month forecast)

Two procedures suggest that the null hypothesis is rather not true for the pound, but true for the franc. For Yen it isn’t that obvious (for 1 month forecast- rejected $H_0$, 3 months forecast - not).

Given that beta are negative and not nearly 1 as assumed, the cases when the null hypothesis is not rejected cause doubts.

As for the bootstrap, it's critical values are all above critical values of chi-squared statistic. According to bootstrap- we should reject the hypothesis for all forecasts (1 and 3 months) and for all the currencies (pound, franc, yen). So, forward rate is NOT a conditionally unbiased predictor of the future spot exchange rate.

We know that bootstrap with large amount of iterations has higher quality of approximation (higher convergence speed) on the finite samples than asymptotic estimates. In our case, there only 234 observations in the sample - not nearly enough for proper asymptotics. Note that all the talks about variance and the quality of approximations take the consistency of (OLS) estimates as given. If the estimates are not consistent for some reason (we misjudged the model, for example), their rate of convergence doesn't matter, because they converge to nowhere => Wald test is simply irrelevant.